<a href="https://colab.research.google.com/github/Gabrielxdf/MachineLearning/blob/main/SBERT_FINE_TUNNING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 4.5 MB/s eta 0:00:00


In [ ]:
from torch.utils.data import DataLoader
import pandas as pd
import re
from sentence_transformers import SentenceTransformer, InputExample, models, evaluation, losses, util
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle

# Obtendo os dados

In [ ]:
url = 'https://drive.google.com/file/d/1Wxc14dsJdUqOGFdauRs-Adw1Hy5Y3fEU/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]

In [ ]:
# Reading and saving to CSV
def get_data_csv():
    df_data = pd.read_csv('data.csv')
    df_data.to_csv('data.csv', index=False, encoding='utf-8')
    return df_data

In [ ]:
df_data = get_data_csv()
df_data

,curriculos,vagas,notas
0,Nome Completo: Ana Paula Santos Formação: B...,Desenvolvedor Full Stack Desenvolver soluções ...,1
1,"Rafael Santos Avenida das Estrelas, 789 - Cida...",Desenvolvedor Full Stack Desenvolver soluções ...,3
2,"Amanda Costa Rua das Flores, 123 - Cidade das ...",Desenvolvedor Full Stack Desenvolver soluções ...,5
3,"Leonardo Santos Avenida das Estrelas, 789 - Ci...",Desenvolvedor Full Stack Desenvolver soluções ...,4
4,"Gabriela Lima Rua dos Programadores, 123 - Cid...",Desenvolvedor Full Stack Desenvolver soluções ...,5
...,...,...,...
6195,"Nome: Ana Santos Endereço: Rua das Análises, 1...","Designer Gráfico Criar layouts, peças gráficas...",1
6196,Nome: Daniel Oliveira Endereço: Avenida dos Ec...,"Designer Gráfico Criar layouts, peças gráficas...",1
6197,Nome: Pedro Almeida Endereço: Rua das Finanças...,"Designer Gráfico Criar layouts, peças gráficas...",1
6198,Nome: Laura Costa Endereço: Avenida da Economi...,"Designer Gráfico Criar layouts, peças gráficas...",1


# Removendo os dígitos

In [ ]:
df_data["cvs"] = df_data["curriculos"].apply(lambda x: re.sub('\d+', '', x))
df_data["jobs"] = df_data["vagas"].apply(lambda x: re.sub('\d+', '', x))

In [ ]:
min_max_scaler = MinMaxScaler()
min_max_scaler.set_output(transform='pandas');

In [ ]:
df_data["scores"] = min_max_scaler.fit_transform(df_data["notas"].values.reshape(-1, 1))
# 1 -> 0.00
# 2 -> 0.25
# 3 -> 0.50
# 4 -> 0.75
# 5 -> 1.00
df_data

,curriculos,vagas,notas
0,Nome Completo: Ana Paula Santos Formação: B...,Desenvolvedor Full Stack Desenvolver soluções ...,0.00
1,"Rafael Santos Avenida das Estrelas, - Cidade ...",Desenvolvedor Full Stack Desenvolver soluções ...,0.50
2,"Amanda Costa Rua das Flores, - Cidade das Art...",Desenvolvedor Full Stack Desenvolver soluções ...,1.00
3,"Leonardo Santos Avenida das Estrelas, - Cidad...",Desenvolvedor Full Stack Desenvolver soluções ...,0.75
4,"Gabriela Lima Rua dos Programadores, - Cidade...",Desenvolvedor Full Stack Desenvolver soluções ...,1.00
...,...,...,...
6195,"Nome: Ana Santos Endereço: Rua das Análises, ...","Designer Gráfico Criar layouts, peças gráficas...",0.00
6196,Nome: Daniel Oliveira Endereço: Avenida dos Ec...,"Designer Gráfico Criar layouts, peças gráficas...",0.00
6197,Nome: Pedro Almeida Endereço: Rua das Finanças...,"Designer Gráfico Criar layouts, peças gráficas...",0.00
6198,Nome: Laura Costa Endereço: Avenida da Economi...,"Designer Gráfico Criar layouts, peças gráficas...",0.00


In [ ]:
# Removing unnecessary columns
df_data.drop(columns=['curriculos', 'vagas', 'notas'])

# Preparando os dados para treinamento

In [ ]:
data_examples = []
for index, row in df_data.iterrows():
    data_examples.append(InputExample(texts=[row['cvs'], row['jobs']], label=row['scores']))

# Splitting the data into 60% for training, 20% for validation and 20% for tests.
data_examples = shuffle(data_examples, random_state=42)
train_index = int(len(data_examples) * 0.6)
val_index = int(len(data_examples) * 0.2)

train_examples = data_examples[:train_index]
val_examples = data_examples[train_index:train_index+val_index]
test_examples = data_examples[train_index+val_index:]

# Creating PyTorch's DataLoader
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=4)

# Treinamento

In [ ]:
checkpoint = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'

word_embedding_model = models.Transformer(checkpoint)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), pooling_mode='mean')
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

train_loss = losses.CosineSimilarityLoss(model)

evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(val_examples, name='sbert')

model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=5, evaluator=evaluator, show_progress_bar=True, save_best_model=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/471M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

...

Iteration:   0%|          | 0/930 [00:00<?, ?it/s]

Iteration:   0%|          | 0/930 [00:00<?, ?it/s]

Iteration:   0%|          | 0/930 [00:00<?, ?it/s]

Iteration:   0%|          | 0/930 [00:00<?, ?it/s]

Iteration:   0%|          | 0/930 [00:00<?, ?it/s]

# Testando um sistema de recomendação de vagas para um currículo

In [ ]:
cv_test = 'Nome: Laura Costa - Objetivo: Busco uma posição como Analista Econômico, onde posso aplicar minha formação acadêmica em Economia e aprimorar minhas habilidades em análise econômica. Formação Acadêmica: Bacharelado em Economia - Universidade Federal de Estado Y (-) Experiência Profissional: Assistente de Análise Econômica - Empresa de Consultoria Econômica LTDA - Cidade Financeira, Estado Y (-Presente) Coleta de dados econômicos. Auxílio na elaboração de relatórios e análises. Habilidades: Conhecimentos intermediários em análise econômica. Familiaridade com ferramentas como Excel e SPSS. Idiomas: Inglês: Avançado Espanhol: Básico'

jobs_test = list(set([test_example.texts[1] for test_example in test_examples]))

In [ ]:
cv_embedding = model.encode(cv_test)
jobs_embedding = [model.encode(vaga) for vaga in jobs_test]
similarity_score = util.cos_sim(cv_embedding, jobs_embedding)

In [ ]:
# Finding the pairs cv-vacancy with the highest cosine similarity score
pairs = []
for index, score in enumerate(similarity_score[0]):
    pairs.append({"index": index, "score": score})

# Sort the pairs by scores in descending order
pairs = sorted(pairs, key=lambda x: x["score"], reverse=True)

In [ ]:
print(f' CV: {cv_test} \n\n')
for pair in pairs[0:5]:
    print(f' Job: {jobs_test[pair["index"]]} \n Predicted similarity score after fine-tuning: {pair["score"]} \n')

 Currículo: Nome: Laura Costa - Objetivo: Busco uma posição como Analista Econômico, onde posso aplicar minha formação acadêmica em Economia e aprimorar minhas habilidades em análise econômica. Formação Acadêmica: Bacharelado em Economia - Universidade Federal de Estado Y (-) Experiência Profissional: Assistente de Análise Econômica - Empresa de Consultoria Econômica LTDA - Cidade Financeira, Estado Y (-Presente) Coleta de dados econômicos. Auxílio na elaboração de relatórios e análises. Habilidades: Conhecimentos intermediários em análise econômica. Familiaridade com ferramentas como Excel e SPSS. Idiomas: Inglês: Avançado Espanhol: Básico 


 Vaga: Analista Econômico Realizar análises econômicas e financeiras; coletar e analisar dados econômicos relevantes; elaborar relatórios e apresentações com as análises realizadas; apoiar na tomada de decisões estratégicas da empresa; acompanhar indicadores econômicos e identificar tendências de mercado. Graduação em Economia ou áreas correlatas